<H1>Clustering Notebook</h1>

This notebook includes the whole process of agglomerative heirarhcical clustering including all visualizations in the presentation. In this notebook is scraping and downloading the data, cleaning it, explaining how clustering works, and picking hyperparameters.

In [ ]:
! pip3 install yfinance -q
! pip3 install numpy -q
! pip3 install seaborn -q
! pip install pandas-datareader -q
! pip install networkx -q
! pip install --upgrade nbformat -q
! pip install sklearn -q
! pip install scipy -q

: 

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from datetime import datetime
''
# this workaround let's us override pandas_datareader with yfinance,
# although we can also use yfinance directly
import yfinance as yf
import pandas_datareader.data as web; yf.pdr_override()
import networkx as nx
import plotly.graph_objects as go
from matplotlib import cm
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.manifold import MDS
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster, cophenet
from sklearn.metrics import silhouette_score
from sklearn import metrics
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import fcluster

: 

<h3>Web Scraping (given)</h3>
scraping from wikipedia and yahoo finance to get companies from S&P 500


our time series is from the past year 2022-2023

In [ ]:
start_date = '2022-01-01'
end_date  = '2022-12-31'

data_dir = '/Users/sofia/Onedrive/Documents/Github/BTT-JPMC-1'
dates = '_{}--{}'.format(start_date, end_date)
dates

: 

so we're getting the symbols from wikipedia

In [ ]:
data    = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S%26P_500_component_stocks')
table_symbol   = data[0]
symbols = list(table_symbol.Symbol.values) #ticker symbols
os.makedirs(data_dir, exist_ok=True)

: 

for each ticker symbol in our list we're going to get the closing price data from 01-01-2022 to 12-31-2022
make that into a dataframe df and store those in our directory

In [ ]:
%%capture
symbols = sorted(symbols)
print("Downloading {} files".format(len(symbols)))
for i, symbol in enumerate(symbols):
    try:
        df = web.get_data_yahoo(symbol, start_date, end_date,)
        # df = web.DataReader(symbol,'yahoo', start_date, end_date)
        df = df[['Adj Close']]
        df.to_csv(os.path.join(data_dir, "{}.csv".format(symbol)))
    except KeyError:
      print("Error for {}".format(symbol))
      pass
print("Stored {} files".format(i+1))

: 

<h2>Pre-processing the data</h2>
Now we have 500 csv files and we want to make one big one, indexed with all dates acorss 2022

In [ ]:
index = pd.date_range(start=start_date, end=end_date, freq='D')     # initialize an empty DateTime Index and so each index is a date over the year
df_price = pd.DataFrame(index=index, columns=symbols)               # initialize empty dataframes

for symbol in symbols:
    symbol_df = pd.read_csv(os.path.join(data_dir, symbol+".csv")).set_index('Date')
    symbol_df.index = pd.to_datetime(symbol_df.index)
    df_price[symbol] = symbol_df['Adj Close']


: 

now we;re going to remove any rows (days) where the stock price is nan, these would be like holidays or weekends

In [ ]:
# Let's drop the dates where all the stocks are NaNs, ie., weekends/holidays where no trading occured
df_price.dropna(how='all', inplace=True)
df_price.dropna(inplace=True, axis=1)
True in pd.isna(df_price)

: 

now we're going to find the price percentage change from day to day with the closing price. This is called the daily returns. we're using daily returns because if we use the raw price data our results will just clump all the big companies (with high stock prices) together, what we're looking for is to see which companies move similarly in the market, not ones that are similalrly priced

In [ ]:
df_price_pct = df_price.pct_change()[1:]
df_price_pct.to_csv(os.path.join(data_dir, "prices_pct.csv"), index_label='date')
df_price_pct = df_price.pct_change().dropna(how='all')
df_price_pct.head()

: 

<h2>Outlier detour</h2>
This is not explored in our presentation but if we had more time we would look into the affect outliers have on our data. There seems to be some days and companies where the returns are abnormally high or low

below shows which companies are outliers across the whole year

In [ ]:
df_price_plotting = df_price_pct
df_2 = df_price_plotting.mean(axis=0)
avg_per_company = list(df_2.values)
plt.boxplot(avg_per_company)
plt.title('average price pct change per company 2022-2023')
plt.ylabel('price pct change averaged over the past year')
plt.show()
df_2.describe()

: 

below shows which days in the time series are outliers

In [ ]:
df_price_plotting['average'] = df_price_plotting.mean(axis=1)
#average across all companies per day
# Creating plot
plt.boxplot(df_price_plotting['average'])
# show plot
plt.title('average price pct change per day')
plt.ylabel('price pct change averaged over the past year')
plt.show()

: 

now this is an overall graph showing how every company on the index did

In [ ]:
plt.title('S&P price change histories during march 2022 (percent)')
plt.plot(df_price_pct, linewidth=0.10)
plt.show()

: 

In [ ]:
df_price_plotting = df_price['2022-04-01':'2022-04-30']
plt.title('S&P price change april 2022')
plt.plot(df_price_plotting['MSFT'], linewidth=1, color="red", label="Microsoft")
plt.plot(df_price_plotting['ADSK'], linewidth=1, color="blue", label="Autodesk")
plt.plot(df_price_plotting['VRSK'], linewidth=1, color="green", label="Verisk")
plt.xticks(rotation=45, ha='right')
plt.ylabel("Closing Price")
plt.legend()
plt.show()

: 

In [ ]:
df_price_plotting = df_price_pct['2022-04-01':'2022-04-30']
plt.title('S&P price change april 2022')
plt.plot(df_price_plotting['MSFT'], linewidth=1, color="red", label="Microsoft")
plt.plot(df_price_plotting['ADSK'], linewidth=1, color="blue", label="Hilton WW")
plt.plot(df_price_plotting['VRSK'], linewidth=1, color="green", label="Verisk")
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.show()

: 

In [ ]:
print('daily returns correlation between autodesk and microsoft', df_price_pct.corr().loc['MSFT', 'ADSK'])
print('daily returns correlation between autodesk and verisk ',df_price_pct.corr().loc['ADSK', 'VRSK'])


: 

now we're going to take those returns and make the dataframe into a correlation matrix. showing how the daily returns of each company is related to the other 500, we want companies that move similarly in the market to be highly correlated

In [ ]:
price_corr = df_price_pct.corr()
price_corr.head()

: 

In [ ]:
sns.heatmap(abs(price_corr)).set_title("Correlation heatmap for Price Changes")

: 

: 

showing how correlation is related to sector

the green companies are utulities sector while orange are real estate companies

In [ ]:
columns = ['CMS', 'DTE', 'LNT', 'SO', 'WEC', 'CPT', 'UDR', 'ESS', 'FRT', 'MAA']
plotting_df = df_price_pct[columns]
plotting_df = plotting_df.corr()
heatmap = sns.heatmap(abs(plotting_df), cmap="Blues")

# Define the companies to be colored differently
green_companies = ['CMS', 'DTE', 'LNT', 'SO', 'WEC']
red_companies = ['CPT', 'UDR', 'ESS', 'FRT', 'MAA']

# Customize the text color on the x-axis
for i, tick_label in enumerate(heatmap.get_xticklabels()):
    company = columns[i]
    if company in green_companies:
        tick_label.set_color("green")
    elif company in red_companies:
        tick_label.set_color("orange")

# Customize the text color on the y-axis
for i, tick_label in enumerate(heatmap.get_yticklabels()):
    company = columns[i]
    if company in green_companies:
        tick_label.set_color("green")
    elif company in red_companies:
        tick_label.set_color("orange")

# Set the title and show the plot
heatmap.set_title("Daily Returns Heatmap For Utilities Industry vs Real Estate")
plt.show()

: 

<h1>Clustering</h1>
heirarhcical agglomerative clustering is a bottom up unsupervised learning approach that creates clusters and visualizes them in a tree-like structure (called dendrogram) this method is useful if you don't know the number of clusters you want beforehand

<h3>Clustering Data</h3>
1-abs(df_price_pct.corr().values)

is a lot to unpack lol. df_price_pct.corr() is the correlation matrix of daily returns for all 500 companies related to each other. We then find the absolute value of those numbers because a strong negative correlation is just as important as a strong positive correlation. We're doing 1- all of that because heirarichical clustering is typically made for distance matrices not similarity matrices. heirarhcial clustering will group together two nodes if they're distance is lower, because that means the two nodes are closer together, to acheive this with similarity we have to take the inverse to highly correlated companies will have a lower number, meaning they are "closer" together

our clustering is based off of the below dissimilarity matrix

In [ ]:
dissimilarity = 1- abs(df_price_pct.corr().values)

#diff types of linkage determine how the clusters are merged 
# single - merge clusters closest together , 
# complete - merge clusters furthest
#ward linkage attempts to minimize variance within clusters after being merged
#single linkage maximized the cophenetic correlation the others were around .34 and .38

: 

using our dissimilarity matrix we create a linkage matrix (Z)
The linkage matrix is what clusters our data points together. Each row in the matrix describes a merge of two clusters, 4 values in each row. the first two values are the indeces of the two datapoints merged, the third value is the disimilarity value they were merged at and the 4th value is how many clusters were merged (always 2)

linkage method is how you calculate which companies are "closer" together in the dissimilariry matrix. Each time a merge is performed the dissimilarity matrix is updated with new values from the merged cluster to all the remaining data points.
<b>Single linkage</b> - recalculates the dissimiarlity (or distance) matrix using the minimum distance between clusters or points
<b>complete linkage</b> - recalculates the distance matrix using the maximum distance between clusters or points
<b>Average linkage</b> - recalculates the distance matrix using the average distance between clusters
<b>Ward linkage</b> - recalculates distance based on how merging those points will increase the sum of squares. The point is to create clusters that minimize variance and are very compact

In [ ]:
Z = linkage(squareform(dissimilarity), method="ward")


: 

once the linkage matrix is made (Z) you can visualize the results with a dendrogram

In [ ]:
dendrogram = sch.dendrogram(Z)
plt.xlabel('Data Points')
plt.ylabel('dissimilarity')
plt.title('Dendrogram')
plt.show()

: 

this dendrogram shows you where all 500 points are merged together at (xaxis is all companies) but to actually get the clusters you need to choose a threshold to cut the dendrogram at. If you cut the above dendrogram at 2.9 dissimialrity there would be 4 clusters, if you cut at 4.5 there will be 2 clusters etc etc

In [ ]:
labels = fcluster(Z, 3.5, criterion='distance')
num_clusters = len(set(labels))


unique_labels = np.unique(labels)
cluster_sizes = [np.sum(labels == label) for label in unique_labels]

print('there are ', num_clusters, ' clustres and these are the sizes of each', cluster_sizes)


: 

In [ ]:
df_labels = pd.DataFrame({'Company': df_price_pct.columns, 'Cluster_Label': labels})


: 

<h2>There are two hyperparameters: number of clusters and linkage</h2>

to find the linkage we looked for linkage methods that would provide a balanced number of clusters so not [499, 1, 1] but maybe [300, 100, 100].
Based off of theory and domain knowledge we can assume the ward linkage would be best

so at various thresholds (number of clusters), I used different linkage methods and found the distributions of nodes the clusters. In theory if the standard deviation of these spreads is smaller then that means the sizes of the clusters are less extreme and more balanced.

finding balanced clusters is very important because if you have [498,1,1] or [1,1,1,2,1,1,1,1,...1,1,1,1] then its the same as not clustering at all

these thresholds are the thresholds for each linkage method that at which you can get 2-15, 20, & 25 clusters from. All of the linkage methods calculate distance differently si theyre all on different scales. this is why i needed threshold numbers for each

In [ ]:
thresholds_ward = [4.0,3.5, 3.0, 2.24,2.2, 2.1,1.981,1.980, 1.7,1.35,1.2, 1.04] #1.35 is threshold for 15 1.2 is 20 clusters
thresholds_comp = [.999,.995,.990, .965,.96,.94,.91,.87,.84,.80,.76, .721 ] #.80 is threshold for 15, 0.76 is 20
thresholds_single = [.55,.54, .53, .51,.50, .49,.486,.485,.480,.471,.4475,.43 ] #.471 is threshold for 15, 0.4475 for 20 clusters
thresholds_average = [.80,.79,.73,.71,.70, .69,.675,.66,.659 ,.6425,.614,.6] #.6425 is theshold for 15, 0.614 is for 20 clsuters
ward_std = []
comp_std = []
aver_std = []
sing_std = []
num_cluster_plot = []

: 

this large for loop finds out the distribution of clusters for each linkage method at varying thresholds (see above cell)
displays result in bar plot, we want to minimize the standard deviation. low standard deviation means the distributions of the cluster sizes are close together

In [ ]:


for i, j,k,l in zip(thresholds_ward, thresholds_comp, thresholds_single, thresholds_average):
    Z_ward = linkage(squareform(dissimilarity), method="ward", optimal_ordering=True)
    Z_comp = linkage(squareform(dissimilarity), method="complete", optimal_ordering=True)
    Z_sing = linkage(squareform(dissimilarity), method="single", optimal_ordering=True)
    Z_aver = linkage(squareform(dissimilarity), method="average", optimal_ordering=True)
    


    labels_ward = fcluster(Z_ward, i, criterion="distance")
    labels_comp = fcluster(Z_comp, j, criterion="distance")
    labels_sing = fcluster(Z_sing, k, criterion="distance")
    labels_aver = fcluster(Z_aver, l, criterion="distance")

    unique_labels_ward = np.unique(labels_ward)
    cluster_sizes_ward = [np.sum(labels_ward == label) for label in unique_labels_ward]

    unique_labels_comp = np.unique(labels_comp)
    cluster_sizes_comp = [np.sum(labels_comp == label) for label in unique_labels_comp]

    unique_labels_sing = np.unique(labels_sing)
    cluster_sizes_sing = [np.sum(labels_sing == label) for label in unique_labels_sing]

    unique_labels_aver = np.unique(labels_aver)
    cluster_sizes_aver = [np.sum(labels_aver == label) for label in unique_labels_aver]


    # Calculate the standard deviation for each method you want the lowest standard devation to make the clusters balanced
    ward_std_dev = np.std(cluster_sizes_ward)
    comp_std_dev = np.std(cluster_sizes_comp)
    ward_std.append(ward_std_dev)
    comp_std.append(comp_std_dev)
    num_cluster_plot.append(max(labels_ward))

    sing_std_dev = np.std(cluster_sizes_sing)
    aver_std_dev = np.std(cluster_sizes_aver)
    sing_std.append(sing_std_dev)
    aver_std.append(aver_std_dev)
    

    print('For ward linkage there are', max(labels_ward), 'clusters and these are the sizes of each', cluster_sizes_ward, ' the standard deviation is ', ward_std_dev)
    print('For complete linkage there are', max(labels_comp), 'clusters and these are the sizes of each', cluster_sizes_comp, ' the standard deivation is ', comp_std_dev)
    print('For single linkage there are', max(labels_sing), 'clusters and these are the sizes of each', cluster_sizes_sing, ' the standard deivation is ', sing_std_dev)
    print('\n')
    
bar_width = 0.15
opacity = 0.8
index = np.arange(len(thresholds_ward))

plt.bar(index, ward_std, bar_width, label='Ward clusters Std', alpha=opacity)
plt.bar(index + bar_width, comp_std, bar_width, label='Complete clusters Std', alpha=opacity)
plt.bar(index+bar_width+bar_width,sing_std, bar_width, label = "Single clusters std", alpha = opacity)
plt.bar(index+bar_width+bar_width+bar_width, aver_std, bar_width, label="Average clusters std", alpha = opacity)



plt.xlabel('Number of Clusters')
plt.ylabel('Standard Deviation')
plt.title('Standard Deviations of cluster distributions for diff linkage methods')
plt.xticks(index + bar_width / 2, [str(clust) for clust in num_cluster_plot])
plt.legend()

plt.tight_layout()
plt.show()



: 

the above bar plot shows that no matter how many clusters are chosen, ward always gives the most balanced distributino of clsuter sizes

so we'll use ward linkage to make our linkage matrix

<h2>We have linkage, lets choose a threshold/h2>

we're going to use the metric silhouette score to find the best threshold to cut our dendrogram at. I'm going to plot it and find where silhouette score is the highest

silhouette score is a metric that values compact and separated clusters so small clusters that are far apart. we want a high silhouette score

In [ ]:
thresholds = np.arange(0.1, 4.5, 0.1)


#im going to store the results of this for loop into a dataframe so i can compare everything better
columns = ['threshold','Silhouette', 'num clusters']
price_corr_eval = pd.DataFrame(columns=columns)
        
for t in thresholds:
        Z = linkage(squareform(dissimilarity), method = "ward")
        labels = fcluster(Z, t, criterion='distance')
        silhouette_avg = silhouette_score(dissimilarity, labels, metric='precomputed')
        #silhouette_scores.append(silhouette_avg)

        c, _ = cophenet(Z, squareform(dissimilarity))
        #cophenet_correlations.append(c)
        ch = calinski_harabasz_score(dissimilarity, labels)
        num_clusters = max(labels)
        new_row = [t, silhouette_avg, num_clusters]
        price_corr_eval.loc[len(price_corr_eval)] = new_row
        #print(f"Silhouette Score for threshold {t}: {silhouette_avg} and cophenet corr for link method {l}: {c}")
price_corr_eval.head()

: 

In [ ]:
plt.plot(price_corr_eval['threshold'], price_corr_eval['Silhouette'])
plt.xticks(np.arange(0, 5.5, 0.5))
# Add labels and title
plt.xlabel('number of clusters')
plt.ylabel('silhouette score')
plt.title('num clust vs silhouette score for ward linkage')

: 

In [ ]:
plt.plot(price_corr_eval['num clusters'], price_corr_eval['Silhouette'])

# Add labels and title
plt.xlabel('number of clusters')
plt.ylabel('silhouette score')
plt.title('num clust vs silhouette score for ward linkage from 0-500')
plt.show()

: 